In [ ]:
import torch
import matplotlib.pyplot as plt
import non_local_boxes
import numpy as np
from IPython.display import clear_output   # in order to clear the print output

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
W = [0.,0.,1.,1.,1.,1.,0.,0.,0.,1.,1.,0.,1.,0.,0.,0.,0.,1.,1.,0.,1.,0.,0.,1.,0.,1.,1.,0.,0.,1.,1.,0.]
non_local_boxes.utils.wiring_to_functions(W)

# Definitions

## Test a wiring in a segment and in a triangle

In [ ]:
def test_wiring_segment(W, Box1, Box1_name, Box2, Box2_name, box_grid_size, max_box_power, threshold):
    # W is a vector of size 32
    n = non_local_boxes.evaluate.nb_columns
    W = torch.t(W.repeat(n, 1))
    # Now W is a 32xn tensor

    X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()
    triangle = plt.Polygon(X[:3,:], color="snow")
    plt.gca().add_patch(triangle)
    
    for i in range(box_grid_size+1):
        alpha = i/box_grid_size
        P = alpha*Box1 + (1-alpha)*Box2   # P is a 4x4 matrix
        color_point = "orangered"

        Q=non_local_boxes.utils.matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor 
        value = float(non_local_boxes.evaluate.h_flat(Q))
        #print(value)
        if value > threshold:
                color_point = (0, 0.1, 0.1)
        else:
            if value != 0.75:
                for l in range(max_box_power):
                    if color_point != "orangered":
                        break
                    Q=non_local_boxes.evaluate.R(W, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]
                    value = float(non_local_boxes.evaluate.h_flat(Q))
                    if value > threshold:
                        color_point = (0, 0.1*(1-l/max_box_power)+1*(l/max_box_power), 0.1*(1-l/max_box_power)+1*(l/max_box_power))

        plt.plot(X[0,0]*alpha + X[1,0]*(1-alpha), X[0,1]*alpha + X[1,1]*(1-alpha), 'o', markersize=3, color=color_point)
                    
    plt.text(X[0,0], X[0,1]+0.1, Box1_name, horizontalalignment='center')
    plt.text(X[1,0]+0.1, X[1,1], Box2_name, verticalalignment='center')
    plt.show()


In [ ]:
def proportion_blue_points_in_a_segment(W, Box1, Box3, box_grid_size, max_box_power, threshold):
    
    number_blue_points = 0

    for i in range(box_grid_size+1):
        alpha = i/box_grid_size
        P = alpha*Box1 + (1-alpha)*Box3   # P is a 4x4 matrix
        color_point = "orangered"

        Q=non_local_boxes.utils.matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor 
        value = float(non_local_boxes.evaluate.h_flat(Q))
        #print(value)
        if value > threshold:
            color_point = (0, 0.1, 0.1)
            number_blue_points += 1
        else:
            if value != 0.75:
                for l in range(max_box_power):
                    if color_point != "orangered":
                        break
                    Q=non_local_boxes.evaluate.R(W, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]
                    value = float(non_local_boxes.evaluate.h_flat(Q))
                    if value > threshold:
                        color_point = (0, 0.1*(1-l/max_box_power)+1*(l/max_box_power), 0.1*(1-l/max_box_power)+1*(l/max_box_power))
                        number_blue_points += 1

    return number_blue_points/box_grid_size

In [ ]:
def test_wiring_triangle(W, Box1, Box1_name, Box2, Box2_name, Box3, Box3_name, box_grid_size, max_box_power, threshold, Details_Are_Printed):
    # W is a vector of size 32
    n = non_local_boxes.evaluate.nb_columns
    W = torch.t(W.repeat(n, 1))
    # Now W is a 32xn tensor

    # Check the proportion of blue points in the top segment -> stop if there is no enough blue points.
    if proportion_blue_points_in_a_segment(W, Box1, Box3, box_grid_size, 21, threshold)<0.39:
        if Details_Are_Printed:
            print("-- Not collapsing:  ", Box1_name," -- ", Box3_name)
        return ""

    X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()
    triangle = plt.Polygon(X[:3,:], color="snow")
    plt.gca().add_patch(triangle)
    
    for i in range(box_grid_size+1):
        for j in range(box_grid_size-i+1):
            alpha, beta = i/box_grid_size, j/box_grid_size
            P = alpha*Box1 + beta*Box2 + (1-alpha-beta)*Box3   # P is a 4x4 matrix
            color_point = "orangered"

            Q=non_local_boxes.utils.matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor 
            value = float(non_local_boxes.evaluate.h_flat(Q))
            #print(value)
            if value > threshold:
                    color_point = (0, 0.1, 0.1)
            else:
                #if value > 0.75:
                for l in range(max_box_power):
                    if color_point != "orangered":
                        break
                    Q=non_local_boxes.evaluate.R(W, non_local_boxes.utils.tensor_to_matrix(Q), P)[:,:,:,:,0]
                    value = float(non_local_boxes.evaluate.h_flat(Q))
                    if value > threshold:
                        color_point = (0, 0.1*(1-l/max_box_power)+1*(l/max_box_power), 0.1*(1-l/max_box_power)+1*(l/max_box_power))

            plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                    
    plt.text(X[0,0], X[0,1]+0.1, Box1_name, horizontalalignment='center')
    plt.text(X[1,0]+0.1, X[1,1], Box2_name, verticalalignment='center')
    plt.text(X[2,0]+0.1, X[2,1], Box3_name, verticalalignment='center')
    plt.show()

In [ ]:
def test_wiring_triangle_with_left_multiplication(W, Box1, Box1_name, Box2, Box2_name, Box3, Box3_name, box_grid_size, max_box_power, threshold):
    # W is a vector of size 32
    n = non_local_boxes.evaluate.nb_columns
    W = torch.t(W.repeat(n, 1))
    # Now W is a 32xn tensor

    # # Check the proportion of blue points in the top segment -> stop if there is no enough blue points.
    # if proportion_blue_points_in_a_segment(W, Box1, Box3, box_grid_size, 21, threshold)<0.39:
    #     if Details_Are_Printed:
    #         print("-- Not collapsing:  ", Box1_name," -- ", Box3_name)
    #     return ""

    X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()
    triangle = plt.Polygon(X[:3,:], color="snow")
    plt.gca().add_patch(triangle)
    
    for i in range(box_grid_size+1):
        for j in range(box_grid_size-i+1):
            alpha, beta = i/box_grid_size, j/box_grid_size
            P = alpha*Box1 + beta*Box2 + (1-alpha-beta)*Box3   # P is a 4x4 matrix
            color_point = "orangered"

            Q=non_local_boxes.utils.matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor 
            value = float(non_local_boxes.evaluate.h_flat(Q))
            #print(value)
            if value > threshold:
                    color_point = (0, 0.1, 0.1)
            else:
                #if value > 0.75:
                for l in range(max_box_power):
                    if color_point != "orangered":
                        break
                    Q=non_local_boxes.evaluate.R(W, P, non_local_boxes.utils.tensor_to_matrix(Q))[:,:,:,:,0]
                    value = float(non_local_boxes.evaluate.h_flat(Q))
                    if value > threshold:
                        color_point = (0, 0.1*(1-l/max_box_power)+1*(l/max_box_power), 0.1*(1-l/max_box_power)+1*(l/max_box_power))

            plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                    
    plt.text(X[0,0], X[0,1]+0.1, Box1_name, horizontalalignment='center')
    plt.text(X[1,0]+0.1, X[1,1], Box2_name, verticalalignment='center')
    plt.text(X[2,0]+0.1, X[2,1], Box3_name, verticalalignment='center')
    plt.show()

In [ ]:
def test_wiring_triangle_with_pairwise_multiplication(W, Box1, Box1_name, Box2, Box2_name, Box3, Box3_name, box_grid_size, max_box_power, threshold):
    # W is a vector of size 32
    n = non_local_boxes.evaluate.nb_columns
    W = torch.t(W.repeat(n, 1))
    # Now W is a 32xn tensor

    # # Check the proportion of blue points in the top segment -> stop if there is no enough blue points.
    # if proportion_blue_points_in_a_segment(W, Box1, Box3, box_grid_size, 21, threshold)<0.39:
    #     if Details_Are_Printed:
    #         print("-- Not collapsing:  ", Box1_name," -- ", Box3_name)
    #     return ""

    X = torch.tensor([[1,3], [4,1], [4, 5]]).numpy()
    triangle = plt.Polygon(X[:3,:], color="snow")
    plt.gca().add_patch(triangle)
    
    for i in range(box_grid_size+1):
        for j in range(box_grid_size-i+1):
            alpha, beta = i/box_grid_size, j/box_grid_size
            P = alpha*Box1 + beta*Box2 + (1-alpha-beta)*Box3   # P is a 4x4 matrix
            color_point = "orangered"

            Q=non_local_boxes.utils.matrix_to_tensor(torch.clone(P))  # Q is a 2x2x2x2 tensor 
            value = float(non_local_boxes.evaluate.h_flat(Q))
            #print(value)
            if value > threshold:
                    color_point = (0, 0.1, 0.1)
            else:
                #if value > 0.75:
                for l in range(max_box_power):
                    if color_point != "orangered":
                        break
                    Q=non_local_boxes.evaluate.R(W, non_local_boxes.utils.tensor_to_matrix(Q), non_local_boxes.utils.tensor_to_matrix(Q))[:,:,:,:,0]
                    value = float(non_local_boxes.evaluate.h_flat(Q))
                    if value > threshold:
                        color_point = (0, 0.1*(1-l/max_box_power)+1*(l/max_box_power), 0.1*(1-l/max_box_power)+1*(l/max_box_power))

            plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)
                    
    plt.text(X[0,0], X[0,1]+0.1, Box1_name, horizontalalignment='center')
    plt.text(X[1,0]+0.1, X[1,1], Box2_name, verticalalignment='center')
    plt.text(X[2,0]+0.1, X[2,1], Box3_name, verticalalignment='center')
    plt.show()

## Test

In [ ]:
W = torch.tensor([0., 0., 1., 1.,              # f_1(x, a_2) = x
            0., 0., 1., 1.,              # g_1(y, b_2) = y
            0., 0., 0., 1.,              # f_2(x, a_1) = a_1*x
            0., 0., 0., 1.,              # g_2(y, b_1) = b_1*y
            0., 1., 1., 0., 0., 1., 1., 0.,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
            0., 1., 1., 0., 0., 1., 1., 0.   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
            ])

# Set the number of wirings to 1

test_wiring_triangle_with_left_multiplication(
    W = W,
    Box1=non_local_boxes.utils.PR,
    Box1_name = "PR",
    Box2=non_local_boxes.utils.P_0,
    Box2_name = "$P_0$",
    Box3=non_local_boxes.utils.P_1,
    Box3_name = "$P_1$",
    box_grid_size = 60,
    max_box_power = 15,
    threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6
)

In [ ]:
W = torch.tensor([0., 0., 1., 1.,              # f_1(x, a_2) = x
            0., 0., 1., 1.,              # g_1(y, b_2) = y
            0., 0., 0., 1.,              # f_2(x, a_1) = a_1*x
            0., 0., 0., 1.,              # g_2(y, b_1) = b_1*y
            0., 1., 1., 0., 0., 1., 1., 0.,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
            0., 1., 1., 0., 0., 1., 1., 0.   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
            ])

# Set the number of wirings to 1

test_wiring_triangle_with_pairwise_multiplication(
    W = W,
    Box1=non_local_boxes.utils.PR,
    Box1_name = "PR",
    Box2=non_local_boxes.utils.P_0,
    Box2_name = "$P_0$",
    Box3=non_local_boxes.utils.P_1,
    Box3_name = "$P_1$",
    box_grid_size = 60,
    max_box_power = 15,
    threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6
)

In [ ]:
W = torch.tensor([0., 0., 1., 1.,              # f_1(x, a_2) = x
            0., 0., 1., 1.,              # g_1(y, b_2) = y
            0., 0., 0., 1.,              # f_2(x, a_1) = a_1*x
            0., 0., 0., 1.,              # g_2(y, b_1) = b_1*y
            0., 1., 1., 0., 0., 1., 1., 0.,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
            0., 1., 1., 0., 0., 1., 1., 0.   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
            ])

# Set the number of wirings to 1

test_wiring_triangle(
    W = W,
    Box1=non_local_boxes.utils.PR,
    Box1_name = "PR",
    Box2=non_local_boxes.utils.P_0,
    Box2_name = "$P_0$",
    Box3=non_local_boxes.utils.P_1,
    Box3_name = "$P_1$",
    box_grid_size = 60,
    max_box_power = 15,
    threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6,
    Details_Are_Printed=False
)

# Last definition

In [ ]:
PR = non_local_boxes.utils.PR
PL = non_local_boxes.utils.P_L
PNL = non_local_boxes.utils.P_NL

W_BS09 = torch.tensor([0., 0., 1., 1.,              # f_1(x, a_2) = x
            0., 0., 1., 1.,              # g_1(y, b_2) = y
            0., 0., 0., 1.,              # f_2(x, a_1) = a_1*x
            0., 0., 0., 1.,              # g_2(y, b_1) = b_1*y
            0., 1., 1., 0., 0., 1., 1., 0.,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
            0., 1., 1., 0., 0., 1., 1., 0.   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
            ])

In [ ]:
# Set the number of wirings to 1

def Test_Wiring(W_list, Fixed_box_in_drawings, Fixed_box_name, known_collapsing_W, Details_Are_Printed, compare_with_known_wirings=True):
    # W_list is given as a list here

    if non_local_boxes.evaluate.nb_columns != 1:
        print("WARNING: please change the number of wirings to 1.")
        return " "

    W = torch.tensor(W_list)*1.
    if Details_Are_Printed:
        print("------ Number of known wirings: ", len(known_collapsing_W))
    print("\nW = ",W,"\n")
    if compare_with_known_wirings:
        for i in range(len(known_collapsing_W)):
            if torch.all(W == torch.tensor(known_collapsing_W[i])):
                print(" !!!  This wiring is already known: ",i+1,"th/ ",len(known_collapsing_W),"  !!!")
                return " "
    
    threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6

    # test_wiring_segment(
    #     W = W,
    #     Box1=PR,
    #     Box1_name = "PR",
    #     Box2 = Fixed_box_in_drawings,
    #     Box2_name = Fixed_box_name,
    #     box_grid_size = 60,
    #     max_box_power = 50,
    #     threshold = (3 + float(torch.sqrt(torch.tensor(6))))/6
    # )    

    n = non_local_boxes.evaluate.nb_columns
    p = proportion_blue_points_in_a_segment(torch.t(W.repeat(n, 1)), PR, Fixed_box_in_drawings, 40, 40, threshold)
    texte = ""
    if Details_Are_Printed:
        texte ="------ "
    print(texte,"Proportion of blue points in the segment PR --", Fixed_box_name, ":   ", 100*p, "%")

    if p>0.4:
        boxes_to_be_tested = non_local_boxes.utils.boxes_to_be_tested
        for box_number in boxes_to_be_tested:
            mu, nu, sigma, tau = box_number
            if PL(mu,nu,sigma,tau).tolist() != Fixed_box_in_drawings.tolist():
                test_wiring_triangle(
                    W = W,
                    Box1=PR,
                    Box1_name = "PR",
                    Box2 = Fixed_box_in_drawings,
                    Box2_name = Fixed_box_name,
                    Box3=PL(mu,nu,sigma,tau),
                    Box3_name = "PL"+str(mu)+str(nu)+str(sigma)+str(tau),
                    box_grid_size = 60,
                    max_box_power = 20,
                    threshold = threshold,
                    Details_Are_Printed = Details_Are_Printed
                )
        
        test_wiring_triangle(
            W = W,
            Box1=PR,
            Box1_name = "PR",
            Box2=PL(0,0,0,0),
            Box2_name = "$P_0$",
            Box3=(PR + non_local_boxes.utils.PRprime)/2,
            Box3_name = "(PR+PR') / 2",
            box_grid_size = 60,
            max_box_power = 20,
            threshold = threshold,
            Details_Are_Printed = Details_Are_Printed
        )

    if Details_Are_Printed:
        print("---------- FINISHED ----------")

# Let's Go!

In [ ]:
known_collapsing_W = non_local_boxes.utils.known_collapsing_W

W = [0.,1.,0.,1.,0.,1.,1.,1.,1.,1.,0.,0.,0.,0.,1.,1.,0.,0.,1.,1.,0.,0.,1.,1.,0.,1.,1.,0.,1.,0.,0.,1.]

non_local_boxes.utils.wiring_to_functions(W)
print("")

Test_Wiring(
    W,
    PL(1,0,0,0),
    "PL0010",
    known_collapsing_W,
    Details_Are_Printed = True,
    compare_with_known_wirings=False
)

---
---
---

In [ ]:
i=0

known_collapsing_W = non_local_boxes.utils.known_collapsing_W

for C in [[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]]:        # TO BE IMPROVED
    for D in [[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]]:    # TO BE IMPROVED
        for A in [[0.,0.,1.,1.], [1.,1.,0.,0.]]:
            for B in [[0.,0.,1.,1.], [1.,1.,0.,0.]]:
                for E in [[0., 1., 1., 0.], [1., 0., 0., 1.]]:
                    for F in [[0., 1., 1., 0.], [1., 0., 0., 1.]]:
                        for G in [[0., 1., 1., 0.], [1., 0., 0., 1.]]:
                            for H in [[0., 1., 1., 0.], [1., 0., 0., 1.]]:
                                i += 1
                                print(" ")
                                print("--------------   TEST NUMBER ", i, "  --------------")
                                print(" ")

                                W = A+B+C+D+E+F+G+H # concatenation of A, B, C, D, E, F, G H

                                Test_Wiring(
                                    W,
                                    PL(0,0,0,0),
                                    "P_0",
                                    known_collapsing_W, 
                                    False
                                )

---
---
---

In [ ]:
# Check that all known wirings are different.

known_collapsing_W = non_local_boxes.utils.known_collapsing_W

N = len(known_collapsing_W)
print("Number of known wirings: ",N)
for i in range(N-1):
    for j in range(i+1,N):
        if known_collapsing_W[i] == known_collapsing_W[j]:
            print("W_",i+1," and W_",j+1," are the same")
            assert(False)
print("-- finished --")

In [ ]:
torch.tensor(known_collapsing_W[65])

In [ ]:
torch.tensor(known_collapsing_W[66])